In [ ]:
%%HTML
<style>
    body {
        --vscode-font-family: "Noto Serif"
    }
</style>

# Trie (Prefix Tree)

A Trie is a tree-like data structure optimized for storing and querying keys that are sequences of tokens—most commonly strings as sequences of characters. Each edge represents one token, and a path from the root to a node spells out a prefix; a special end marker indicates a complete key.

Key properties and benefits:
- Fast prefix queries: efficient for autocomplete, spell-check suggestions, IP routing, and dictionary operations.
- Deterministic behavior: no hashing; lookups follow the key's tokens.
- Space–time trade-off: often uses more memory than hash tables due to many nodes but shines when many keys share prefixes.

Typical operations (n = length of key, p = length of prefix):
- Insert: O(n)
- Search (exact): O(n)
- Starts-with (prefix existence): O(p)
- Enumerate keys by prefix: O(p + k) where k is total length of reported keys

Common use cases:
- Autocomplete and search-as-you-type
- Spell checkers and word games (e.g., Boggle)
- Longest-prefix match (e.g., routing tables)
- Dictionary implementation with ordered traversal by prefix

In [ ]:
use std::collections::HashMap;
use std::hash::Hash;

// TrieNode represents a node in the trie
#[derive(Debug, Clone)]
struct TrieNode<T> {
    children: HashMap<T, TrieNode<T>>,
    terminal: bool,
    value: Option<String>, // Using String for simplicity
}

impl<T> TrieNode<T>
where
    T: Clone + Eq + Hash,
{
    fn new() -> Self {
        TrieNode {
            children: HashMap::new(),
            terminal: false,
            value: None,
        }
    }
}

// Generic Trie over sequences of tokens
#[derive(Debug)]
pub struct Trie<T> {
    root: TrieNode<T>,
    size: usize,
}

impl<T> Trie<T>
where
    T: Clone + Eq + Hash,
{
    // Create a new trie
    pub fn new() -> Self {
        Trie {
            root: TrieNode::new(),
            size: 0,
        }
    }

    // Get the number of keys
    pub fn len(&self) -> usize {
        self.size
    }

    // Check if empty
    pub fn is_empty(&self) -> bool {
        self.size == 0
    }

    // Insert a key with optional value
    pub fn insert(&mut self, key: &[T], value: Option<String>) {
        let mut node = &mut self.root;
        for token in key {
            node = node.children.entry(token.clone()).or_insert_with(TrieNode::new);
        }
        if !node.terminal {
            node.terminal = true;
            self.size += 1;
        }
        node.value = value;
    }

    // Check if key exists as complete entry
    pub fn contains(&self, key: &[T]) -> bool {
        if let Some(node) = self.traverse(key) {
            node.terminal
        } else {
            false
        }
    }

    // Get value associated with key
    pub fn get(&self, key: &[T]) -> Option<&String> {
        if let Some(node) = self.traverse(key) {
            if node.terminal {
                return node.value.as_ref();
            }
        }
        None
    }

    // Check if any key exists with given prefix
    pub fn starts_with(&self, prefix: &[T]) -> bool {
        self.traverse(prefix).is_some()
    }

    // Get all keys that start with prefix
    pub fn keys_with_prefix(&self, prefix: &[T]) -> Vec<Vec<T>> {
        if let Some(node) = self.traverse(prefix) {
            let mut results = Vec::new();
            let mut path = prefix.to_vec();
            self.dfs_keys(node, &mut path, &mut results);
            results
        } else {
            Vec::new()
        }
    }

    // Find longest key that is prefix of query
    pub fn longest_prefix_of(&self, query: &[T]) -> Vec<T> {
        let mut node = &self.root;
        let mut best_end = None;
        
        for (i, token) in query.iter().enumerate() {
            if let Some(child) = node.children.get(token) {
                node = child;
                if node.terminal {
                    best_end = Some(i);
                }
            } else {
                break;
            }
        }
        
        if let Some(end) = best_end {
            query[..=end].to_vec()
        } else {
            Vec::new()
        }
    }

    // Traverse to node following key path
    fn traverse(&self, key: &[T]) -> Option<&TrieNode<T>> {
        let mut node = &self.root;
        for token in key {
            node = node.children.get(token)?;
        }
        Some(node)
    }

    // DFS to collect keys starting from node
    fn dfs_keys(&self, node: &TrieNode<T>, path: &mut Vec<T>, results: &mut Vec<Vec<T>>) {
        if node.terminal {
            results.push(path.clone());
        }
        
        for (token, child) in &node.children {
            path.push(token.clone());
            self.dfs_keys(child, path, results);
            path.pop();
        }
    }
}

impl<T> Default for Trie<T>
where
    T: Clone + Eq + Hash,
{
    fn default() -> Self {
        Self::new()
    }
}

// CharTrie - convenience wrapper for string tries
#[derive(Debug)]
pub struct CharTrie {
    trie: Trie<char>,
}

impl CharTrie {
    // Create new character trie
    pub fn new() -> Self {
        CharTrie {
            trie: Trie::new(),
        }
    }

    // Insert word
    pub fn insert_word(&mut self, word: &str, value: Option<String>) {
        let chars: Vec<char> = word.chars().collect();
        self.trie.insert(&chars, value);
    }

    // Check if word exists
    pub fn contains_word(&self, word: &str) -> bool {
        let chars: Vec<char> = word.chars().collect();
        self.trie.contains(&chars)
    }

    // Get words with prefix
    pub fn words_with_prefix(&self, prefix: &str) -> Vec<String> {
        let prefix_chars: Vec<char> = prefix.chars().collect();
        let key_chars = self.trie.keys_with_prefix(&prefix_chars);
        
        key_chars.into_iter()
                 .map(|chars| chars.into_iter().collect())
                 .collect()
    }

    // Longest prefix of word
    pub fn longest_prefix_of_word(&self, query: &str) -> String {
        let query_chars: Vec<char> = query.chars().collect();
        let result_chars = self.trie.longest_prefix_of(&query_chars);
        result_chars.into_iter().collect()
    }

    // Check prefix existence
    pub fn starts_with(&self, prefix: &str) -> bool {
        let prefix_chars: Vec<char> = prefix.chars().collect();
        self.trie.starts_with(&prefix_chars)
    }

    // Get size
    pub fn len(&self) -> usize {
        self.trie.len()
    }

    // Check if empty
    pub fn is_empty(&self) -> bool {
        self.trie.is_empty()
    }
}

impl Default for CharTrie {
    fn default() -> Self {
        Self::new()
    }
}

## Examples

Below are quick examples showing trie usage for autocomplete and prefix matching.

In [ ]:
// Character trie demo
let mut ct = CharTrie::new();
let words = vec!["to", "tea", "ted", "ten", "in", "inn"];
for word in &words {
    ct.insert_word(word, None);
}

println!("Size: {}", ct.len());
println!("Contains 'tea': {}", ct.contains_word("tea"));
println!("Contains 'te': {}", ct.contains_word("te"));
println!("Starts with 'te': {}", ct.starts_with("te"));
println!("Words with prefix 'te': {:?}", ct.words_with_prefix("te"));
println!("Longest prefix of 'tendril': '{}'", ct.longest_prefix_of_word("tendril"));

In [ ]:
// Testing with generic trie for integers
let mut int_trie: Trie<i32> = Trie::new();
int_trie.insert(&[1, 2, 3], Some("path123".to_string()));
int_trie.insert(&[1, 2, 4], Some("path124".to_string()));
int_trie.insert(&[1, 3], Some("path13".to_string()));

println!("Integer trie size: {}", int_trie.len());
println!("Contains [1,2,3]: {}", int_trie.contains(&[1, 2, 3]));
println!("Starts with [1,2]: {}", int_trie.starts_with(&[1, 2]));
println!("Keys with prefix [1]: {:?}", int_trie.keys_with_prefix(&[1]));